# **Great Salt Lake Area Analysis**
Author: Raj Saha

This notebook processes satellite imagery of the Great Salt Lake to analyze changes in lake area over time
See full article at: https://raj-saha.github.io/great-salt-lake-area/


In [1]:
import cv2
import numpy as np
import os


In [2]:

def preprocess_image(img_path):
    """
    Loads and resizes image to ensure dimensions are multiples of 4.
    
    Args:
        img_path: Path to input image file
        
    Returns:
        Preprocessed image with dimensions that are multiples of 4
    """
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    new_height = (height // 4) * 4
    new_width = (width // 4) * 4
    
    if new_height < height or new_width < width:
        img = img[:new_height, :new_width]
    elif new_height > height or new_width > width:
        img = np.pad(img, ((0, new_height - height), (0, new_width - width), (0, 0)), 'constant')
    return img

def save_blue_channel_images(input_dir, output_dir):
    """
    Extracts and saves blue channel from all images in input directory.
    
    Args:
        input_dir: Directory containing input images
        output_dir: Directory to save blue channel images
    """
    os.makedirs(output_dir, exist_ok=True)
    filenames = os.listdir(input_dir)
    image_filenames = [f for f in filenames if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    for image_filename in image_filenames:
        image_path = os.path.join(input_dir, image_filename)
        image = cv2.imread(image_path)
        blue_channel = image[:, :, 0]  # Extract blue channel
        output_filename = os.path.splitext(image_filename)[0] + '_blue.jpg'
        output_path = os.path.join(output_dir, output_filename)
        cv2.imwrite(output_path, blue_channel)

def apply_green_outline(input_dir, threshold=128):
    """
    Calculates area of lake in each image by counting pixels above threshold.
    
    Args:
        input_dir: Directory containing blue channel images
        threshold: Pixel intensity threshold for binarization (default 128)
        
    Returns:
        Dictionary mapping frame numbers to pixel counts
    """
    filenames = os.listdir(input_dir)
    image_filenames = [f for f in filenames if f.endswith(('.jpg', '.jpeg', '.png'))]
    pixel_count_dict = {}

    for image_filename in image_filenames:
        image_path = os.path.join(input_dir, image_filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        _, binarized_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
        pixel_count = np.count_nonzero(binarized_image > threshold)

        frame_num = int(image_filename.split('_')[1])
        pixel_count_dict[frame_num] = pixel_count

    return pixel_count_dict


In [3]:
# Define input and output directories
input_dir = '../_frames/'  # Directory containing original image frames
blue_dir = '../_frames/_blue/'  # Directory to store extracted blue channel images

# Process images by extracting blue channel from each frame
save_blue_channel_images(input_dir, blue_dir)

# Calculate area of exposed land in pixels for each frame using threshold=128
# Higher pixel values indicate exposed land, lower values indicate water
area_by_frame = apply_green_outline(blue_dir, threshold=128)

# Sort frames chronologically and create list of (frame, area) tuples
sorted_areas = [(frame, area) for frame, area in sorted(area_by_frame.items())]

In [4]:
# Convert to DataFrame for easier manipulation and display
import pandas as pd
import cv2

# Get total lake area from 1985 mask
mask_lake = cv2.imread('mask_lake.png', cv2.IMREAD_GRAYSCALE)
total_lake_area = np.count_nonzero(mask_lake > 128)

# Create DataFrame
df = pd.DataFrame(sorted_areas, columns=['Frame', 'Area_exposed'])

# Add year column (assuming frames start from 1985)
df['Year'] = df['Frame'].apply(lambda x: 1984 + x)

# Calculate actual lake area by subtracting exposed area from total lake area
df['Area_pixels'] = total_lake_area - df['Area_exposed']

# Calculate area as percentage change from 1985 (first year) area
base_area = df.iloc[0]['Area_pixels']
df['Area_percent'] = ((df['Area_pixels'] - base_area) / base_area * 100).round(1)

df


,Frame,Area_exposed,Year,Area_pixels,Area_percent
0,37,3611,2021,611196,0.0
1,46,4279,2030,610528,-0.1
2,57,4310,2041,610497,-0.1
3,77,9087,2061,605720,-0.9
4,87,25181,2071,589626,-3.5
5,98,46681,2082,568126,-7.0
6,107,69057,2091,545750,-10.7
7,118,95942,2102,518865,-15.1
8,137,96308,2121,518499,-15.2
9,148,108164,2132,506643,-17.1
